# Derivatives of a function with multiple variables

In the last exercise we: 
- implemented the forward kinematic for the Openmanipulator-X
- and implemented a way to solve the inverse kinematic with NLSolve.jl

What did we actually do when we used the NLSolve.jl package? We used an implementation of a root finding method to find the solution to our inverse kinematic problem. We learned in the lecture how to find the root of a set of functions with multiple variables. For example we learned that we can find the root of a set of functions with the Newton method. Can we actually implement the Newton method for multiple variables ourself? 

Well we need to know how to calculate the derivative of a function with multiple variables. In the next exercise we will take a closer look into the Newton Method for multiple variables. For now we will just look at the algorithm of the Newton method for multiple variables. Similar to the one dimensional case we calculate the next $\mathbf{x}$ which is closer to the root with the following calculation rule:

$$
\begin{split}\begin{split}
  \mathbf{x}_{n+1} &= \mathbf{x}_n - \mathbf{J}(\mathbf{x}_n)^{-1}\mathbf{f}(\mathbf{x}_n).
\end{split}\end{split}
$$
The calculation rule for the next $\mathbf{x}$ which is closer to the root is thus iteratively called over and over again. The calculation rule according to the Newton method is therefore:
$$
\mathbf{x}_0 = \text{start value}
$$
$$
\mathbf{x}_{1} = \mathbf{x}_0 - \mathbf{J}(\mathbf{x}_0)^{-1}\mathbf{f}(\mathbf{x}_0)
$$
$$
\mathbf{x}_{2} = \mathbf{x}_{1} - \mathbf{J}(\mathbf{x}_{1})^{-1}\mathbf{f}(\mathbf{x}_{1})
$$
...
$$
\mathbf{x}_{n+1} = \mathbf{x}_n - \mathbf{J}(\mathbf{x}_n)^{-1}\mathbf{f}(\mathbf{x}_n)
$$
We do this until we have reached the desired accuracy, i.e., the desired distance of $\mathbf{f}(\mathbf{x}_{n+1})$ to $\mathbf{0}$.

In the calculation rule we see that we need to calculate the function $\mathbf{f}(\mathbf{x})$, the derivative of the function (the Jacobian $\mathbf{J}(\mathbf{x})$) and the inverse of the Jacobian matrix $\mathbf{J}(\mathbf{x})^{-1}$.

**We are still missing some Important Parts...**

So let's start with estimating the derivative of a function with multiple variables.

# Derivatives of a functions with multiple variables

## Partial Derivatives
In the lecture we have already seen the partial derivative of a function with multiple variables. We can calculate the partial derivative of a function $f(x,y)$ with respect to $x$ by treating $y$ as a constant:
$$
\frac{\partial f}{\partial x} 
$$
We can interpret this as how much $f(x,y)$ changes when $x$ changes. We can do this for all variables of $f$. Hence, we can also calculate the partial derivative of the function $f(x,y)$ with respect to $y$ by treating $x$ as a constant. 

For our example from above $f(x,y) = 0.5*sin(x) + cos(y)$ we would get:
$$
\frac{\partial f(x,y)}{\partial x} = 0.5*cos(x)
$$
for the partial derivative with respect to $x$ and 
$$
\frac{\partial f(x,y)}{\partial y} = -sin(y)
$$
for the partial derivative with respect to $y$.

As an example, let's take a look again at the plot of the lecture were we plotted the partial derivative of $f(x,y)$ with respect to $x$ and $y$ seperately:

In [ ]:
using Revise
include("./PartialDerivativesPlot.jl")
using .PartialDerivativesPlot
using GLMakie

function parabola(x, y)
    return @. x^2 + y^2
end

angle = LinRange(0, 2*pi, 64)
radius = LinRange(0, 1, 16)

x = radius .* cos.(angle')
y = radius .* sin.(angle')
z = parabola(x, y)

tangentPoint=[0.0, 0.5]

with_theme(theme_dark()) do
    fig = Figure(resolution = (1000, 950), fontsize = 22)
    ax = LScene(fig[1,1], show_axis = true)

    sm = surface!(ax, x, y, z; colormap = :viridis, colorrange = (minimum(z), maximum(z)),
        transparency = false)

    # partial derivative w.r.t. x
    PartialDerivativesPlot.directionalbox!(ax, parabola, tangentPoint, :x, tangentColor=GLMakie.Colors.HSL(0, 1, 0.4))

    # partial derivative w.r.t. y
    PartialDerivativesPlot.directionalbox!(ax, parabola, tangentPoint, :y, tangentColor=GLMakie.Colors.HSL(125, 1, 0.4))

    display(fig)
end

## Total Derivatives

The total derivative of a function $f(x,y)$ with respect to $x$ and $y$ is defined as:
$$
d f = \frac{\partial f}{\partial x} {d x} + \frac{\partial f}{\partial y} {d y} =  f_x {d x} + f_y {d y} 
$$

Remember, the total derivative gives us the relationship of changes in the function $f(x,y)$ when $x$ and $y$ change. Hence $d x$ and $d y$ are not variables we can calculate with: $d x \neq \Delta x$. 
We can make an approximation: 
$$
\Delta x \approx \frac{\partial f}{\partial x} {\Delta x} + \frac{\partial f}{\partial y} {\Delta y}
$$

## Chain rule
If we want to calculate the derivative of a function $f(x(t),y(t))$ with respect another variable $t$, where $x(t)$ and $y(t)$ depend on $t$ we can use the chain rule:
$$
\frac{d f}{d t} = \frac{\partial f}{\partial x} \frac{d x}{d t} + \frac{\partial f}{\partial y} \frac{d y}{d t}
$$
This can also be written in a more general way:
$$
\frac{\partial f}{\partial t} = \frac{\partial f}{\partial x} \frac{\partial x}{\partial t} + \frac{\partial f}{\partial y} \frac{\partial y}{\partial t}
$$
This form is also valid for functions with more indepedant variables. 

## Gradient
The gradient of a function $f(x,y)$ is defined as:
$$
\nabla f = \begin{bmatrix}
\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}
\end{bmatrix}
$$
and we can write the total derivative as:
$$
d f = \nabla f \cdot \begin{bmatrix}
d x \\
d y
\end{bmatrix}
$$

## Jacobian Matrix
If we want to calculate the total derivative of a system of equations, we can define a function as $\mathbf{f}= \begin{bmatrix} f_1 \\ f_2 \end{bmatrix}$ and the total derivative of this function is defined as:
$$
d \mathbf{f} = \begin{bmatrix}
\frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} \\
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y}
\end{bmatrix} \cdot \begin{bmatrix}
d x \\
d y
\end{bmatrix}
= \mathbf{J} \cdot \begin{bmatrix}
d x \\
d y
\end{bmatrix}
$$
The matrix $\mathbf{J}$ is called the Jacobian matrix of the function $\mathbf{f}$. The Jacobian matrix is a matrix of all partial derivatives of the function $\mathbf{f}$ with respect to all variables. The columns of the Jacobian matrix are the gradients of the function $\mathbf{f}$ with respect to the variables: 
$$
\mathbf{J} = \begin{bmatrix}
\nabla f_1 \\
 \nabla f_2
\end{bmatrix} = \begin{bmatrix}
\frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} \\
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y}
\end{bmatrix} 
$$


# Derivatives of a functions 
In the lecture, we learned different ways to calculate the derivative of a function:
- finite differences
    - forward difference
    - central difference
- automatic differentiation (AD)
    - forward mode
    - reverse mode

A good explanation for the error in estimating derivatives and automatic differentiation can be found here: https://book.sciml.ai/notes/08-Forward-Mode_Automatic_Differentiation_(AD)_via_High_Dimensional_Algebras/
Some parts of this exercise are based on these lecture notes.

## Automatic Differentiation (AD)
Automatic Differentiation (AD) is a set of techniques to numerically evaluate the derivative of a function. AD is generally more accurate than the finite differences based numerical differentiation we used above. In the lecture we have seen two different methods of AD: the forward mode and the reverse mode (Backpropagation).

### Forward Mode
You have seen in the lecture that the forward mode AD makes use of dual numbers to calculate the derivative of a function. For each value $x$ we create a dual number $x + \epsilon x'$ where $\epsilon$ is a small number and $x'$ is the derivative of $x$. We can use the dual number to calculate the derivative of a function $f(x)$:
$$
f(x + \epsilon) = f(x) + \epsilon f'(x) + \mathcal{O}(\epsilon^2)
$$
As discussed in the lecture we defined $\epsilon^2 = 0$ and got:
$$
f(x + \epsilon) = f(x) + \epsilon f'(x)
$$

To use this in practice we have to implement $f(x)$ to support our dual numbers for each function we want to calculate the derivative. Hence, we have to implement what happens with the derivative $f'(x)$ for each operation we want to support. For example, for the addition of two dual numbers $x + \epsilon x'$ and $y + \epsilon y'$ we get:
$$
(x + \epsilon x') + (y + \epsilon y') = (x + y) + \epsilon (x' + y')
$$ 
We can see that the derivative of the addition is the addition of the derivatives. We can implement this in Julia by overloading the addition operator `+` for dual numbers. We can do this similarly for the other operations.

We can use the package `ForwardDiff.jl` to do this for us. Let's try this out for the function $f(x) = x^2 + sin(x)$. We can use the function `ForwardDiff.derivative` to calculate the derivative of $f(x)$ at $x$:

In [ ]:
import ForwardDiff
f(x) = x^2+sin(x)
df(x) = 2*x+cos(x) # analytical derivative of f to estimate the error

@show # TODO: estimate derivative and error using ForwardDiff.derivative
@show df(1.0);

We can calculate the gradient of a function $f(x,y)$ with respect to $x$ and $y$ using the function `ForwardDiff.gradient`:

In [ ]:
f(x) = x[1]^2+x[1]*x[2]

@show # TODO: estimate the gradient of f at x, y using ForwardDiff.gradient

We can also calculate the Jacobian matrix using the function `jacobian` from `ForwardDiff`. This function takes a function as an argument and returns the Jacobian matrix of the function. Let's try this out for the function 
$$
\mathbf{f(x)} =  \begin{bmatrix}
x_1^2 + x_1*x_2 \\
x_1^2 + x_2^2
\end{bmatrix} 
$$.

In [ ]:
f(x) = [x[1]^2+x[1]*x[2]; 
        x[1]^2+x[2]^2]
x = [1.0, 1.0]

@show # TODO: estimate the Jacobian of f at x using ForwardDiff.jacobian

### Reverse Mode
When we want to compute the gradient of a function $f(x)$ with respect to $x$ we have seen that we can use the forward mode AD. This is a good choice if we have a function $f(x)$ with a small number of variables. If we have a function $f(x)$ with a large number of variables the forward mode AD is not efficient. In this case we can use the reverse mode AD (backpropagation, adjoint technique). In the lecture we have seen that the reverse mode AD uses the chain rule to calculate the gradient of a function $f(x(z), y(z))$ with respect to $z$. We can write the chain rule as:
$$
\frac{\partial f}{\partial z} = \frac{\partial f}{\partial x} \frac{\partial x}{\partial z} + \frac{\partial f}{\partial y} \frac{\partial y}{\partial z}
$$
where $x, y$ are intermediate variables. We can use this to calculate the gradient of a function $f(x(z), y(z))$ with respect to $z$ by calculating the partial derivative of $f(x(z), y(z))$ with respect to each intermediate variable $x$ or $y$ and multiplying it with the partial derivative of $x$ or $y$ with respect to $z$. We can see that we have to calculate the partial derivative of $f$ with respect to each intermediate variable $x,y $ and the partial derivative of $x,y $ with respect to $z$. A popular example for the use of reverse mode AD is in deep learning to calculate the gradient of the loss function with respect to the weights of the neural network. In this case we have a large number of weights and often a scalar loss function. Hence, the reverse mode AD is a good choice. Now we want to take a look at the reverse mode AD in practice. We will use the package `ReverseDiff.jl` to do this. Let's try this out for the function $f(x_1, x_2) = x_1^2 + x_1*x_2$. 

If you are interested in the details of reverse mode AD you can have a look at the lecture slides again or find a good explanation here: https://book.sciml.ai/notes/10-Basic_Parameter_Estimation-Reverse-Mode_AD-and_Inverse_Problems/

In [ ]:
using Pkg
Pkg.add("ReverseDiff")

In [ ]:
import ReverseDiff

f(x) = x[1]^2+x[1]*x[2]
@show # TODO: estimate the gradient of f at x, y using ReverseDiff.gradient

In [ ]:
f(x) = [x[1]^2+x[1]*x[2]; 
        x[1]^2+x[2]^2]
@show # TODO: estimate the Jacobian of f at x using ReverseDiff.jacobian

We can see for the basic usage `ReverseDiff` and `ForwardDiff` are very similar. Which one you should use depends on the function you want to differentiate. It is very well summarized in https://book.sciml.ai: Forward mode computes columns of a Jacobian, while reverse mode computes gradients (rows of a Jacobian). Therefore, the relative efficiency of the two approaches is based on the size of the Jacobian. 
For $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$ with $n$ variables and $m$ functions the Jacobian is a $m \times n$ matrix. So computing each row will be much faster if $m$ is much smaller than $n$. Or in other words if the number of variables is larger than the number of functions, then reverse mode is more efficient. If the number of functions is larger than the number of variables, then forward mode is more efficient. If $m$ and $n$ are relatively equal forward mode is often more efficient since reverse mode has a high overhead compared to forward mode. If you are not sure which approach to use you can try both and benchmark the results.

# Calculation of the end effector velocity based on the joint velocities

We can measure the joint velocities of our Openmanipulator. However, we are interested in the speed of the end effector. We have seen in the lecture that we can calculate the end effector velocity based on the joint velocities using the Jacobian matrix. 

## Estimating the Jacobian matrix of the Openmanipulator-X kinematics
To estimate the velocity of the end effector we need to know the forward kinematics of the Openmanipulator-X and estimate the derivative of the forward kinematics. 



In [ ]:
import Pkg
Pkg.activate("../Exercise 3/OpenMEnv") 
using Revise
include("../OpenManipulatorLib/OpenManipulatorKinematics.jl")
using .OpenManipulatorKineamtics

Estimate the jacobian matrix of the forward kinematics for our robot arm using automatic differentiation. We can use the function `ForwardDiff.jacobian` to calculate the Jacobian matrix of the forward kinematics. 

In [ ]:
using ForwardDiff

function openmanipulator_jacobian(q)
    # Define a function that returns the position of the end effector
    f(q) = # TODO: calculate the position of the end effector using OpenManipulatorKinematics.forward_kinematics 

    # Calculate the Jacobian matrix using ForwardDiff.jacobian
    J = # TODO: calculate the Jacobian matrix using ForwardDiff.jacobian

    return J
end

Now we can use the function `ReverseDiff.jacobian` to calculate the Jacobian matrix of the forward kinematics.

In [ ]:
using ReverseDiff

function openmanipulator_jacobian_rev_AD(q)
    # Define a function that returns the position of the end effector
    f(q) = # TODO: calculate the position of the end effector using OpenManipulatorKinematics.forward_kinematics

    # Calculate the Jacobian matrix using ReverseDiff.jacobian
    J = # TODO: calculate the Jacobian matrix using ReverseDiff.jacobian

    return J
end

Now estimate the jacobian matrix using finite differences:

In [ ]:
import FiniteDifferences

function openmanipulator_jacobian_finiteDiff(q)
    # Define a function that returns the position of the end effector
    f(q) = # TODO: calculate the position of the end effector using OpenManipulatorKinematics.forward_kinematics
    
    # Calculate the Jacobian matrix FiniteDifferences 
    J = # TODO: calculate the Jacobian matrix using FiniteDifferences.jacobian

    return J
end

Now, lets compare the time and results of the methods. What do you observe?

> Call all cells a second time to see if the results change? Why do you think the results change? 

In [ ]:
q = [0.1, 0.0, 0.0, 0.1]

In [ ]:
@time openmanipulator_jacobian(q)

In [ ]:
@time openmanipulator_jacobian_rev_AD(q)

In [ ]:
@time openmanipulator_jacobian_finiteDiff(q)

Now let's compare again where we run the calculation several times for each approach:

In [ ]:
function jacobian_random_benchmark(jacobian_function)
    for i in 1:10000
        q_init = rand(4)
        jacobian_function(q_init)
    end
end

In [ ]:
@time jacobian_random_benchmark(openmanipulator_jacobian)
@time jacobian_random_benchmark(openmanipulator_jacobian_rev_AD)
@time jacobian_random_benchmark(openmanipulator_jacobian_finiteDiff)

We can also estimate the performance of the different methods by using the `BenchmarkTools.jl` package. We can use the `@benchmark` macro to measure the time and memory efficiency of the calculation. Let's try this out for the different methods:

In [ ]:
Pkg.add("BenchmarkTools")
using BenchmarkTools

In [ ]:
@benchmark openmanipulator_jacobian(q)

In [ ]:
@benchmark openmanipulator_jacobian_rev_AD(q)

In [ ]:
@benchmark openmanipulator_jacobian_finiteDiff(q)

## Calculating of the end effector velocity using the Jacobian matrix

Erlier we said that based on the total derivative we can derive the rule for differentiating our system of equations $f_1(x,y)$ and $f_2(x,y)$ with respect to the variable $t$ when $x$ and $y$ are functions of $t$:
$$
\frac{d}{dt} \begin{bmatrix}
f_1(x,y) \\
f_2(x,y)
\end{bmatrix} = \begin{bmatrix}
\frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} \\
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y}
\end{bmatrix} \begin{bmatrix}
\frac{\partial x}{\partial t} \\
\frac{\partial y}{\partial t}
\end{bmatrix}
= \mathbf{J} \begin{bmatrix}
\frac{\partial x}{\partial t} \\
\frac{\partial y}{\partial t}
\end{bmatrix}
$$

Based on this rule we can estimate the end effector velocity using the Jacobian matrix. We can use the following estimation:
$$
v = \mathbf{J}(q) * \mathbf{q}_v = \mathbf{J}(q) * \begin{bmatrix}
\frac{\partial q_1}{\partial t} \\
\frac{\partial  q_2}{\partial t} \\
\frac{\partial  q_3}{\partial t} \\
\frac{\partial  q_4}{\partial t} \\
\end{bmatrix} 
\approx J(q) * \begin{bmatrix}
\Delta q_1 \\
\Delta q_2 \\
\Delta q_3 \\
\Delta q_4 \\
\end{bmatrix} 
$$

where $v$ is the end effector velocity, $\mathbf{J}(q)$ is the Jacobian matrix evaluated at the current joint angles $q$, and $\mathbf{q}_v$ is the joint velocity vector, which we can estimate with the joint approximate velocities $\Delta q_1$, $\Delta q_2$, $\Delta q_3$, and $\Delta q_4$.

Implement the `endeffector_velocity` function that uses the Jacobian matrix to calculate the end effector velocity. The function should takes in the joint angles `q` and joint velocity vector `q_v`, and returns the end effector velocity `v` calculated using the Jacobian matrix. Use the `openmanipulator_jacobian` function you implemented to calculate the Jacobian matrix.

In [ ]:
function endeffector_velocity(q, q_v)
    # Calculate the Jacobian matrix at the current joint angles
    J = # TODO: calculate the Jacobian matrix of the Openmanipulator 

    # Calculate the end effector velocity using the Jacobian matrix
    v = # TODO: calculate the end effector velocity using the Jacobian matrix and the joint velocities

    return v
end

Let's test the function with the following joint angles and joint velocities:

In [ ]:
q_v = [0.0, 0.0, 0.0, 0.0]
v = endeffector_velocity(q, q_v)

## Applying the end effector velocity calculation to the Openmanipulator-X

- let's start the Openmanipulator and subscrib to the joint states
- Calculate the end effector velocity using the joint velocities and the Jacobian matrix
- Do this by adding a new function which is the new callback function for the joint states subscriber and which calls the `endeffector_velocity` function
- print the end effector velocity to the console while you move the Openmanipulator

In [ ]:
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat

In [ ]:
srcdir = "../open_manipulator_description/urdf/"
urdf = joinpath(srcdir, "open_manipulator.urdf")
mechanism = parse_urdf(urdf)
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));

# Calculating of the end effector velocity based on the joint velocities

In [ ]:
state = mvis.state
joint_state = mvis.state.q
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
zero_velocity!(state)

In [ ]:
function no_control!(torques::AbstractVector, t, state::MechanismState)
    qs = state.q[1:4] 
    q_vs = state.v[1:4]
    v = endeffector_velocity(qs, q_vs)
    println("v: $v")
    total_velocity = norm(v)
    println("total_velocity: $total_velocity")
end;

In [ ]:
final_time = 2.0
ts, qs, vs = simulate(mvis.state, final_time, no_control!; Δt=1e-2);

# Controling the Openmanipulator-X and calculating the end effector velocity

In [ ]:
state = mvis.state
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
zero_velocity!(state)

In [ ]:
function joint_position_control(q_state, v_state, q_desired)
    kp = 10.0
    v_damping = 0.1
    max_torque = 15.0

    torque = kp * (q_desired - q_state) - v_damping * v_state
    torque = clamp.(torque, -max_torque, max_torque)

    return torque
end;

In [ ]:
om_joints = joints(state.mechanism)

function simple_joint_control!(torques::AbstractVector, t, state::MechanismState)
    target_joint_number = 1
    positions = [0.5, 0.0, 0.0, 0.0]

    for joint in om_joints
        torques[velocity_range(state, joint)] .= 0.0
    end
    for joint_number in 1:length(om_joints)
        if joint_number <= 4
            torques[velocity_range(state, om_joints[joint_number])] .= joint_position_control(state.q[joint_number], state.v[joint_number], positions[joint_number])
        else
            state.v[joint_number] = 0.0
        end
    end

    qs = state.q[1:4]
    q_vs = state.v[1:4]
    v = endeffector_velocity(qs, q_vs)
    total_velocity = norm(v)
    println("total_velocity: $total_velocity")
end;

In [ ]:
final_time = 5.0
ts, qs, vs = simulate(mvis.state, final_time, simple_joint_control!; Δt=1e-2);
animate(mvis, ts, qs);
joint_state = mvis.state.q